In [100]:
import pandas as pd
import numpy as np

In [101]:
pd.set_option('display.max_columns', None)

In [102]:
df = pd.read_csv('data_wrangling_medical_2022_u4587426.csv', dtype=str)

In [103]:
def fix_email(row):
    email = str(row['email'])
    if '@' not in email:
        try:
            pre = email.split('.com')[0]
            post = email.split('.com')[1]
            pre_dot = pre.rfind('.')
            domain = pre[pre_dot+1:]
            user = pre[:pre_dot]
            return f'{user}@{domain}.com{post}'
        except IndexError:
            return 'unknown'
    else:
        return email

In [104]:
for col in df.columns:
    df[col] = df[col].str.strip()
    df[col] = df[col].str.replace('  ', ' ')

In [105]:
# Clean medicare and phone
cols = ['medicare_number', 'phone']
for col in cols:
    df[col] = df[col].str.replace(' ', '')

In [106]:
# Fix '24' in timestamp - should be 00
cols = ['consultation_timestamp']
for col in cols:
    df[col] = df[col].str.replace('t24', 't00')

In [107]:
from datetime import datetime
df['consult_ts'] = [datetime.strptime(x, '%Y-%m-%dt%H:%M%z') for x in df['consultation_timestamp']]

In [108]:
df['consult_year'] = [x.year for x in df['consult_ts']]
df['consult_month'] = [x.month for x in df['consult_ts']]
df['consult_day'] = [x.day for x in df['consult_ts']]
df['consult_hour'] = [x.hour for x in df['consult_ts']]
df['consult_minute'] = [x.minute for x in df['consult_ts']]
df['consult_dt'] = [x.date() for x in df['consult_ts']]

In [109]:
df['email_fix'] = df.apply(lambda row: fix_email(row), axis=1)

In [110]:
display(df)

,rec_id,ssn,first_name,middle_name,last_name,gender,age_at_consultation,birth_date,medicare_number,street_address,suburb,postcode,state,phone,email,marital_status,height,weight,bmi,blood_pressure,cholesterol_level,smoking_status,clinical_notes,consultation_timestamp,consult_ts,consult_year,consult_month,consult_day,consult_hour,consult_minute,consult_dt,email_fix
0,rec-72052,i116534747,donald,lydell,manning,f,13,4/6/2004,16713221912,1 gisborne road goodmans,bacchus marsh,3340,vic,0382242324,donald.manning@hotmail.com,NaN,139,67,34,71,134,0,basal-cell-carcinoma---skin-leasion-to-left-as...,2017-11-15t10:10+00:00,2017-11-15 10:10:00+00:00,2017,11,15,10,10,2017-11-15,donald.manning@hotmail.com
1,rec-66635,g196248364,george,allen,morrison,m,10,12/3/2010,98232867912,1 leura lane watersley cottage,hamilton,3300,vic,NaN,spfefrcmeq@mail.com,NaN,120,50,34,69,97,0,depressed-mood,2020-07-20t10:22+00:00,2020-07-20 10:22:00+00:00,2020,7,20,10,22,2020-07-20,spfefrcmeq@mail.com
2,rec-73551,e111796351,stephon,hill,warren,f,28,25/12/1989,78248955711,39 mcnabb loop collier park vlg,como,6152,wa,0871245514,NaN,married,167,141,50,74,129,0,low-energy,2017-02-23t11:13+00:00,2017-02-23 11:13:00+00:00,2017,2,23,11,13,2017-02-23,unknown
3,rec-52311,f113787195,matthew,e,heuer,m,18,3/11/1997,65095255323,5 harrys road rivendell,arthurs creek,3099,vic,0306974454,heuer21@mail.com,not-married,172,66,22,73,128,1,atrial-fibrillation,2015-04-04t14:39+00:00,2015-04-04 14:39:00+00:00,2015,4,4,14,39,2015-04-04,heuer21@mail.com
4,rec-49632,g166722356,melisa,NaN,kellogg,f,59,12/11/1957,89262992512,9 hallett close riverside garden,douglas,4814,qld,0732902798,NaN,married,185,122,35,73,119,0,low-back-pain,2016-06-13t19:26+00:00,2016-06-13 19:26:00+00:00,2016,6,13,19,26,2016-06-13,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,rec-15872,i161581251,gracie,little,ratcliffe,f,6,12/8/2008,90398995912,501 little collins street aprt 701,melbourne,3000,vic,0303794953,gracie.ratcliffe@aol.com,NaN,123,25,16,70,131,0,depression,2014-04-07t23:47+00:00,2014-04-07 23:47:00+00:00,2014,4,7,23,47,2014-04-07,gracie.ratcliffe@aol.com
19996,rec-53302,g152500166,allan,landon,hurdle,m,49,3/8/1968,41489830122,5 barrett drive floreat village,alice springs,0870,nt,0860047509,NaN,married,180,138,42,80,152,0,ringing-ears,2017-02-09t20:48+00:00,2017-02-09 20:48:00+00:00,2017,2,9,20,48,2017-02-09,unknown
19997,rec-90066,g178482871,joshua,michael,munn,f,7,22/4/2008,77378845112,49 ellis road whispering pines,alstonville,2477,nsw,0259633249,joshua.munn@mail.com.au,NaN,112,26,20,76,187,0,cataract-surgery---right-eye,2015-07-06t07:17+00:00,2015-07-06 07:17:00+00:00,2015,7,6,7,17,2015-07-06,joshua.munn@mail.com.au
19998,rec-51517,h169776084,carol,lucille,harms,m,59,1/12/1956,95352801521,360 eusdale road woodlands,yetholme,2795,nsw,NaN,harms.carol@mail.com,married,193,123,33,74,138,0,bronchitis,2015-09-19t05:22+00:00,2015-09-19 05:22:00+00:00,2015,9,19,5,22,2015-09-19,harms.carol@mail.com


In [111]:
# Validate if the suburb entered matches the postcode entered (0 = no, 1 = yes, invalid = postcode doesn't exist)
pc = pd.read_csv('australian_postcodes.csv', dtype=str)
pc = pc[['postcode', 'locality', 'state']]
pc1 = pc.groupby('postcode')['locality'].apply(list)
df = df.replace(np.nan, '', regex=True)

In [112]:
def pcode_valid(row):
    if row['postcode'] != '' and row['suburb'] != '':
        try:
            pcode_src = row['postcode']
            suburb_src = row['suburb']
            suburbs_lookup = [x.lower() for x in pc1[pcode_src]]
            if suburb_src in suburbs_lookup:
                return 1
            else:
                return 0
        except KeyError:
            return 'invalid postcode'

In [113]:
df['pcode_check'] = df.apply(lambda row: pcode_valid(row), axis=1)

In [114]:
pc2 = pc
pc2['key'] = pc.locality + pc.state
pc2['key'] = pc2.key.str.lower()

In [115]:
display(pc2)

,postcode,locality,state,key
0,0200,ANU,ACT,anuact
1,0200,Australian National University,ACT,australian national universityact
2,0800,DARWIN,NT,darwinnt
3,0801,DARWIN,NT,darwinnt
4,0804,PARAP,NT,parapnt
...,...,...,...,...
18436,9013,BRISBANE,QLD,brisbaneqld
18437,9015,BRISBANE,QLD,brisbaneqld
18438,9464,NORTHGATE MC,QLD,northgate mcqld
18439,9726,GOLD COAST MC,QLD,gold coast mcqld


In [116]:
def suburb_valid(row):
    if row['postcode'] != '' and row['suburb'] != '' and row['state'] != '':
        pcode_src = row['postcode']
        suburb_src = row['suburb']
        state_src = row['state']
        src_key = suburb_src + state_src
        try:
            postcode_lookup = pc2.loc[pc2['key'] == src_key]['postcode'].values[0]
            if pcode_src == postcode_lookup:
                return 1
            else:
                return 0
        except IndexError:
            return 'invalid locality'

In [117]:
df['suburb_check'] = df.apply(lambda row: suburb_valid(row), axis=1)

In [118]:
# Convert date columns
cols = ['birth_date']
for col in cols:
    df[col+'_dt'] = [datetime.strptime(x, '%d/%m/%Y') for x in df[col]]

In [119]:
display(df)

,rec_id,ssn,first_name,middle_name,last_name,gender,age_at_consultation,birth_date,medicare_number,street_address,suburb,postcode,state,phone,email,marital_status,height,weight,bmi,blood_pressure,cholesterol_level,smoking_status,clinical_notes,consultation_timestamp,consult_ts,consult_year,consult_month,consult_day,consult_hour,consult_minute,consult_dt,email_fix,pcode_check,suburb_check,birth_date_dt
0,rec-72052,i116534747,donald,lydell,manning,f,13,4/6/2004,16713221912,1 gisborne road goodmans,bacchus marsh,3340,vic,0382242324,donald.manning@hotmail.com,,139,67,34,71,134,0,basal-cell-carcinoma---skin-leasion-to-left-as...,2017-11-15t10:10+00:00,2017-11-15 10:10:00+00:00,2017,11,15,10,10,2017-11-15,donald.manning@hotmail.com,1,1,2004-06-04
1,rec-66635,g196248364,george,allen,morrison,m,10,12/3/2010,98232867912,1 leura lane watersley cottage,hamilton,3300,vic,,spfefrcmeq@mail.com,,120,50,34,69,97,0,depressed-mood,2020-07-20t10:22+00:00,2020-07-20 10:22:00+00:00,2020,7,20,10,22,2020-07-20,spfefrcmeq@mail.com,1,1,2010-03-12
2,rec-73551,e111796351,stephon,hill,warren,f,28,25/12/1989,78248955711,39 mcnabb loop collier park vlg,como,6152,wa,0871245514,,married,167,141,50,74,129,0,low-energy,2017-02-23t11:13+00:00,2017-02-23 11:13:00+00:00,2017,2,23,11,13,2017-02-23,unknown,1,1,1989-12-25
3,rec-52311,f113787195,matthew,e,heuer,m,18,3/11/1997,65095255323,5 harrys road rivendell,arthurs creek,3099,vic,0306974454,heuer21@mail.com,not-married,172,66,22,73,128,1,atrial-fibrillation,2015-04-04t14:39+00:00,2015-04-04 14:39:00+00:00,2015,4,4,14,39,2015-04-04,heuer21@mail.com,1,1,1997-11-03
4,rec-49632,g166722356,melisa,,kellogg,f,59,12/11/1957,89262992512,9 hallett close riverside garden,douglas,4814,qld,0732902798,,married,185,122,35,73,119,0,low-back-pain,2016-06-13t19:26+00:00,2016-06-13 19:26:00+00:00,2016,6,13,19,26,2016-06-13,unknown,1,0,1957-11-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,rec-15872,i161581251,gracie,little,ratcliffe,f,6,12/8/2008,90398995912,501 little collins street aprt 701,melbourne,3000,vic,0303794953,gracie.ratcliffe@aol.com,,123,25,16,70,131,0,depression,2014-04-07t23:47+00:00,2014-04-07 23:47:00+00:00,2014,4,7,23,47,2014-04-07,gracie.ratcliffe@aol.com,1,1,2008-08-12
19996,rec-53302,g152500166,allan,landon,hurdle,m,49,3/8/1968,41489830122,5 barrett drive floreat village,alice springs,0870,nt,0860047509,,married,180,138,42,80,152,0,ringing-ears,2017-02-09t20:48+00:00,2017-02-09 20:48:00+00:00,2017,2,9,20,48,2017-02-09,unknown,1,1,1968-08-03
19997,rec-90066,g178482871,joshua,michael,munn,f,7,22/4/2008,77378845112,49 ellis road whispering pines,alstonville,2477,nsw,0259633249,joshua.munn@mail.com.au,,112,26,20,76,187,0,cataract-surgery---right-eye,2015-07-06t07:17+00:00,2015-07-06 07:17:00+00:00,2015,7,6,7,17,2015-07-06,joshua.munn@mail.com.au,1,1,2008-04-22
19998,rec-51517,h169776084,carol,lucille,harms,m,59,1/12/1956,95352801521,360 eusdale road woodlands,yetholme,2795,nsw,,harms.carol@mail.com,married,193,123,33,74,138,0,bronchitis,2015-09-19t05:22+00:00,2015-09-19 05:22:00+00:00,2015,9,19,5,22,2015-09-19,harms.carol@mail.com,1,1,1956-12-01


In [99]:
df.to_csv('medical_clean_2.csv', index=False)